# 🎯 YouTube 검색 + 자막 추출 + Gemini 요약 + Notion 연동 (완전판)

**두 가지 모드를 지원하는 완전한 YouTube AI 요약 시스템**

### 🔥 지원 기능:
- ✅ **모드 1**: 직접 URL 입력으로 요약
- ✅ **모드 2**: 검색어로 인기 동영상 자동 선별 + 일괄 요약
- ✅ **YouTube Data API v3**: 검색, 인기도 정렬, 메타데이터 수집
- ✅ **자막 추출**: 최신 API로 다양한 언어 지원
- ✅ **Gemini AI 요약**: 고품질 구조화 요약
- ✅ **Notion 자동 저장**: 깔끔한 페이지 생성

In [4]:
# 패키지 import
import asyncio
import json
import logging
import os
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Optional, Union
from urllib.parse import urlparse
import re
import time

# 웹 크롤링 및 HTTP
import requests
from bs4 import BeautifulSoup
import validators

# Google GenAI
import google.generativeai as genai

# YouTube Data API
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Notion API
from notion_client import Client
import numpy as np
import pandas as pd

# 유튜브 (최신 API)
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound

# 환경변수
from dotenv import load_dotenv
load_dotenv()

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ 모든 패키지가 성공적으로 로드되었습니다!")

✅ 모든 패키지가 성공적으로 로드되었습니다!


## 🔑 API 키 및 환경 설정

In [5]:
# 환경 변수 설정
NOTION_TOKEN = os.getenv("NOTION_TOKEN")
PARENT_PAGE_ID = os.getenv("notion_parent_page")  # 상위 페이지 ID
NOTION_DATABASE_ID = os.getenv("NOTION_DATABASE_ID")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")  # Gemini API 키
YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_KEY")  # YouTube Data API v3 키

print("🔑 환경 변수 상태:")
print(f"  • NOTION_TOKEN: {'✅ 설정됨' if NOTION_TOKEN else '❌ 설정 필요'}")
print(f"  • PARENT_PAGE_ID: {'✅ 설정됨' if PARENT_PAGE_ID else '❌ 설정 필요'}")
print(f"  • NOTION_DATABASE_ID: {'✅ 설정됨' if NOTION_DATABASE_ID else '❌ 설정 필요'}")
print(f"  • GEMINI_API_KEY: {'✅ 설정됨' if GEMINI_API_KEY else '❌ 설정 필요'}")
print(f"  • YOUTUBE_API_KEY: {'✅ 설정됨' if YOUTUBE_API_KEY else '❌ 설정 필요 (검색 기능용)'}")

if not all([NOTION_TOKEN, PARENT_PAGE_ID, GEMINI_API_KEY]):
    print("⚠️ 필수 환경 변수가 설정되지 않았습니다!")
    print("💡 .env 파일에 다음 항목들을 설정해주세요:")
    print("   - NOTION_TOKEN=your_notion_token")
    print("   - notion_parent_page=your_parent_page_id")
    print("   - GEMINI_API_KEY=your_google_ai_key")
    print("   - YOUTUBE_API_KEY=your_youtube_data_api_key (검색 기능용)")

🔑 환경 변수 상태:
  • NOTION_TOKEN: ✅ 설정됨
  • PARENT_PAGE_ID: ✅ 설정됨
  • NOTION_DATABASE_ID: ✅ 설정됨
  • GEMINI_API_KEY: ✅ 설정됨
  • YOUTUBE_API_KEY: ✅ 설정됨


## 🎥 YouTube 검색 및 메타데이터 수집 클래스

In [6]:
class YouTubeSearcher:
    """YouTube Data API v3를 사용한 검색 및 메타데이터 수집"""
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        if api_key:
            try:
                self.youtube = build('youtube', 'v3', developerKey=api_key)
                print("✅ YouTube Data API v3 초기화 성공!")
            except Exception as e:
                print(f"❌ YouTube API 초기화 실패: {str(e)}")
                self.youtube = None
        else:
            self.youtube = None
            print("⚠️ YouTube API 키가 없어 검색 기능을 사용할 수 없습니다.")
    
    def search_popular_videos(self, query: str, max_results: int = 3) -> List[Dict]:
        """
        검색어로 인기 동영상 검색
        
        Args:
            query: 검색어 (예: "파이썬 프로그래밍")
            max_results: 가져올 최대 결과 수
            
        Returns:
            List[Dict]: 인기 동영상 정보 리스트
        """
        if not self.youtube:
            return []
        
        try:
            print(f"🔍 '{query}' 검색 중... (최대 {max_results}개)")
            
            # 검색 실행
            search_response = self.youtube.search().list(
                q=query,
                part='id,snippet',
                maxResults=max_results * 3,  # 여유분으로 더 많이 검색
                order='viewCount',  # 조회수 기준 정렬
                type='video',
                videoDuration='medium',  # 중간 길이 (4분-20분)
                videoCaption='closedCaption',  # 자막 있는 영상만
                relevanceLanguage='ko'  # 한국어 관련성 우선
            ).execute()
            
            videos = []
            video_ids = []
            
            # 비디오 ID 수집
            for search_result in search_response.get('items', []):
                video_id = search_result['id']['videoId']
                video_ids.append(video_id)
            
            if not video_ids:
                print("❌ 검색 결과가 없습니다.")
                return []
            
            # 상세 정보 가져오기
            videos_response = self.youtube.videos().list(
                part='snippet,statistics,contentDetails',
                id=','.join(video_ids)
            ).execute()
            
            for video in videos_response.get('items', []):
                video_info = {
                    'video_id': video['id'],
                    'title': video['snippet']['title'],
                    'channel': video['snippet']['channelTitle'],
                    'description': video['snippet']['description'][:200] + '...',
                    'published_at': video['snippet']['publishedAt'],
                    'view_count': int(video['statistics'].get('viewCount', 0)),
                    'like_count': int(video['statistics'].get('likeCount', 0)),
                    'comment_count': int(video['statistics'].get('commentCount', 0)),
                    'duration': video['contentDetails']['duration'],
                    'url': f"https://www.youtube.com/watch?v={video['id']}",
                    'thumbnail': video['snippet']['thumbnails']['high']['url']
                }
                videos.append(video_info)
            
            # 조회수 기준으로 정렬
            videos.sort(key=lambda x: x['view_count'], reverse=True)
            
            # 최대 결과 수로 제한
            videos = videos[:max_results]
            
            print(f"✅ {len(videos)}개의 인기 동영상을 찾았습니다!")
            for i, video in enumerate(videos, 1):
                print(f"  {i}. {video['title']} (조회수: {video['view_count']:,})")
            
            return videos
            
        except HttpError as e:
            print(f"❌ YouTube API 오류: {str(e)}")
            return []
        except Exception as e:
            print(f"❌ 검색 오류: {str(e)}")
            return []
    
    def search_multiple_keywords(self, keywords: List[str], videos_per_keyword: int = 3) -> Dict[str, List[Dict]]:
        """
        여러 키워드로 검색
        
        Args:
            keywords: 검색어 리스트 (예: ["파이썬", "C", "자바"])
            videos_per_keyword: 키워드당 가져올 동영상 수
            
        Returns:
            Dict[str, List[Dict]]: 키워드별 동영상 리스트
        """
        results = {}
        
        for keyword in keywords:
            print(f"\n🎯 키워드 '{keyword}' 처리 중...")
            videos = self.search_popular_videos(keyword, videos_per_keyword)
            results[keyword] = videos
            
            # API 제한을 위한 잠시 대기
            time.sleep(1)
        
        return results

print("🔍 YouTubeSearcher 클래스 정의 완료!")

🔍 YouTubeSearcher 클래스 정의 완료!


## 🤖 통합 YouTube 처리 Agent (확장됨)

In [7]:
class EnhancedYouTubeGeminiNotionAgent:
    """검색 기능이 추가된 YouTube + Gemini + Notion 통합 Agent"""
    
    def __init__(self, notion_token: str, parent_page_id: str, google_ai_api_key: str, 
                 youtube_api_key: str = None, database_id: str = None):
        """
        초기화
        
        Args:
            notion_token: Notion API 토큰
            parent_page_id: Notion 상위 페이지 ID
            google_ai_api_key: Google AI API 키
            youtube_api_key: YouTube Data API 키 (검색용)
            database_id: Notion 데이터베이스 ID (선택사항)
        """
        # 기본 설정
        self.notion_token = notion_token
        self.parent_page_id = parent_page_id
        self.database_id = database_id
        
        # Gemini 설정
        genai.configure(api_key=google_ai_api_key)
        self.model = genai.GenerativeModel('gemini-pro')
        
        # YouTube 검색 설정
        self.searcher = YouTubeSearcher(youtube_api_key) if youtube_api_key else None
        
        # Notion Client (optional)
        if database_id:
            self.notion_client = Client(auth=notion_token)
        
        print("🚀 Enhanced YouTube-Gemini-Notion Agent 초기화 완료!")
        if not youtube_api_key:
            print("⚠️ YouTube API 키가 없어 직접 URL 모드만 사용 가능합니다.")
    
    def extract_youtube_video_id(self, url: str) -> str:
        """유튜브 URL에서 비디오 ID 추출"""
        patterns = [
            r'(?:youtube\.com\/watch\?v=|youtu\.be\/)([^&\n?#]+)',
            r'youtube\.com\/embed\/([^&\n?#]+)',
            r'youtube\.com\/v\/([^&\n?#]+)',
            r'youtube\.com\/shorts\/([^&\n?#]+)'
        ]
        
        for pattern in patterns:
            match = re.search(pattern, url)
            if match:
                return match.group(1)
        return None
    
    def extract_youtube_transcript(self, url_or_id: str) -> Dict[str, Union[str, bool]]:
        """YouTube 자막 추출 (URL 또는 ID 지원)"""
        try:
            # URL인지 ID인지 판단
            if 'youtube.com' in url_or_id or 'youtu.be' in url_or_id:
                video_id = self.extract_youtube_video_id(url_or_id)
            else:
                video_id = url_or_id
            
            if not video_id:
                return {'success': False, 'error': '유효하지 않은 YouTube URL 또는 ID입니다.'}
            
            print(f"🔍 비디오 ID: {video_id}")
            
            # 최신 API 사용
            api = YouTubeTranscriptApi()
            
            # 다양한 언어로 시도
            languages_to_try = [['ko'], ['en'], ['a.ko'], ['a.en']]
            
            for languages in languages_to_try:
                try:
                    print(f"🔄 {languages[0]} 언어 자막 시도 중...")
                    fetched_transcript = api.fetch(video_id, languages=languages)
                    
                    # FetchedTranscriptSnippet 객체의 속성 접근
                    content = ' '.join([item.text for item in fetched_transcript])
                    content = re.sub(r'\s+', ' ', content).strip()
                    
                    print(f"✅ 자막 추출 성공! (언어: {languages[0]}, 길이: {len(content)}자)")
                    return {
                        'content': content,
                        'success': True,
                        'language': languages[0],
                        'video_id': video_id,
                        'length': len(content)
                    }
                except Exception as e:
                    print(f"❌ {languages[0]} 실패: {str(e)}")
                    continue
            
            # 사용 가능한 자막 목록 확인
            try:
                print("📋 사용 가능한 자막 목록 확인 중...")
                transcript_list = api.list(video_id)
                
                if len(transcript_list) > 0:
                    first_transcript = transcript_list[0]
                    print(f"🔄 첫 번째 자막 언어로 시도: {first_transcript.language_code}")
                    
                    fetched_transcript = first_transcript.fetch()
                    content = ' '.join([item.text for item in fetched_transcript])
                    content = re.sub(r'\s+', ' ', content).strip()
                    
                    print(f"✅ 자막 추출 성공! (언어: {first_transcript.language_code}, 길이: {len(content)}자)")
                    return {
                        'content': content,
                        'success': True,
                        'language': first_transcript.language_code,
                        'video_id': video_id,
                        'length': len(content)
                    }
                
            except Exception as e:
                print(f"❌ 자막 목록 확인 실패: {str(e)}")
            
            return {'success': False, 'error': '자막을 찾을 수 없거나 네트워크 제한이 있습니다.'}
            
        except Exception as e:
            return {'success': False, 'error': f'오류 발생: {str(e)}'}
    
    async def summarize_with_gemini(self, content: str, custom_prompt: str = '', 
                                  video_info: Dict = None) -> Dict[str, Union[str, bool]]:
        """
        Gemini로 콘텐츠 요약 (비디오 정보 포함)
        
        Args:
            content: 요약할 텍스트
            custom_prompt: 사용자 정의 프롬프트
            video_info: 비디오 메타데이터 (제목, 채널 등)
            
        Returns:
            Dict: {'summary': str, 'success': bool, 'error': str}
        """
        try:
            # 기본 요약 프롬프트
            default_prompt = """
다음은 유튜브 영상의 자막입니다. 이를 학습자를 위해 체계적으로 요약해주세요.

## 📝 요약 구조
1. **영상 정보** - 제목, 채널, 조회수 등 기본 정보
2. **핵심 주제와 목표** - 이 영상이 다루는 주요 내용
3. **주요 개념과 내용** - 핵심 포인트를 3-5개로 정리
4. **실습/예제 내용** - 코드나 실습이 있다면 상세히 기록
5. **학습 포인트와 결론** - 중요한 takeaway

각 섹션을 명확하게 구분하고, 마크다운 형식으로 구조화해서 작성해주세요.
"""
            
            # 프롬프트 선택
            system_prompt = custom_prompt if custom_prompt else default_prompt
            
            # 비디오 정보 추가
            video_context = ""
            if video_info:
                video_context = f"""
## 📺 영상 정보:
- 제목: {video_info.get('title', 'Unknown')}
- 채널: {video_info.get('channel', 'Unknown')}
- 조회수: {video_info.get('view_count', 0):,}회
- 좋아요: {video_info.get('like_count', 0):,}개
"""
            
            # 콘텐츠가 너무 긴 경우 축약
            max_content_length = 25000
            if len(content) > max_content_length:
                content = content[:max_content_length] + "\n\n... (내용이 길어 일부만 요약됩니다)"
                print(f"⚠️ 콘텐츠가 길어서 {max_content_length}자로 축약됨")
            
            # 전체 프롬프트 구성
            full_prompt = f"{system_prompt}{video_context}\n\n## 원본 콘텐츠:\n{content}"
            
            print(f"🧠 Gemini로 요약 생성 중... (콘텐츠 길이: {len(content)}자)")
            
            # Gemini 호출 (오류 수정됨)
            response = await asyncio.get_event_loop().run_in_executor(
                None, 
                lambda: self.model.generate_content(
                    full_prompt,
                    generation_config={
                        'temperature': 0.3,
                        'max_output_tokens': 2048,
                        'top_p': 0.8,
                        'top_k': 40
                    }
                )
            )
            
            if not response.text:
                return {
                    'summary': '',
                    'success': False,
                    'error': 'Gemini에서 응답을 생성하지 못했습니다.'
                }
            
            summary = response.text.strip()
            
            print(f"✅ Gemini 요약 생성 성공! (길이: {len(summary)}자)")
            return {
                'summary': summary,
                'success': True,
                'error': None,
                'tokens_used': len(full_prompt.split()) + len(summary.split())
            }
            
        except Exception as e:
            print(f"❌ Gemini 요약 생성 오류: {str(e)}")
            return {
                'summary': '',
                'success': False,
                'error': f'요약 생성 중 오류가 발생했습니다: {str(e)}'
            }
    
    def create_notion_page_with_summary(self, title: str, summary: str, original_content: str, 
                                       url: str, metadata: dict = None, video_info: Dict = None) -> Dict[str, Union[str, bool]]:
        """
        Notion에 요약된 페이지 생성 (메타데이터 확장)
        """
        try:
            # Notion API 헤더
            headers = {
                "Authorization": f"Bearer {self.notion_token}",
                "Content-Type": "application/json",
                "Notion-Version": "2022-06-28"
            }
            
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            # 제목 설정
            if not title:
                if video_info and video_info.get('title'):
                    title = f"📺 {video_info['title']}"
                else:
                    title = f"YouTube 요약 - {current_time}"
            
            # 새 페이지 생성
            new_page = {
                "parent": {"page_id": self.parent_page_id},
                "properties": {
                    "title": [{"text": {"content": title}}]
                }
            }
            
            page_response = requests.post(
                "https://api.notion.com/v1/pages", 
                headers=headers, 
                data=json.dumps(new_page)
            )
            
            if page_response.status_code != 200:
                return {'success': False, 'error': f'페이지 생성 실패: {page_response.text}'}
            
            page_data = page_response.json()
            page_id = page_data['id']
            page_url = page_data['url']
            
            print(f"✅ Notion 페이지 생성 완료: {page_id}")
            
            # 페이지 콘텐츠 구성
            blocks = []
            
            # 1. YouTube URL 링크
            blocks.append({
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [
                        {"type": "text", "text": {"content": "🔗 원본 영상: "}},
                        {"type": "text", "text": {"content": url, "link": {"url": url}}}
                    ]
                }
            })
            
            # 2. 비디오 정보 (확장됨)
            if video_info:
                info_text = f"📊 채널: {video_info.get('channel', 'Unknown')} | 조회수: {video_info.get('view_count', 0):,}회 | 좋아요: {video_info.get('like_count', 0):,}개"
                blocks.append({
                    "object": "block",
                    "type": "paragraph",
                    "paragraph": {
                        "rich_text": [{"type": "text", "text": {"content": info_text}}]
                    }
                })
            
            # 3. 메타데이터
            if metadata:
                meta_text = f"🔤 자막 언어: {metadata.get('language', 'unknown')} | 자막 길이: {metadata.get('length', 0):,}자 | 생성일: {current_time}"
                blocks.append({
                    "object": "block",
                    "type": "paragraph",
                    "paragraph": {
                        "rich_text": [{"type": "text", "text": {"content": meta_text}}]
                    }
                })
            
            # 4. 구분선
            blocks.append({"object": "block", "type": "divider", "divider": {}})
            
            # 5. AI 요약
            blocks.append({
                "object": "block",
                "type": "heading_1",
                "heading_1": {
                    "rich_text": [{"type": "text", "text": {"content": "🤖 Gemini AI 요약"}}]
                }
            })
            
            # 6. 요약 내용
            summary_chunks = [summary[i:i+2000] for i in range(0, len(summary), 2000)]
            for chunk in summary_chunks:
                blocks.append({
                    "object": "block",
                    "type": "paragraph",
                    "paragraph": {
                        "rich_text": [{"type": "text", "text": {"content": chunk}}]
                    }
                })
            
            # 7. 구분선
            blocks.append({"object": "block", "type": "divider", "divider": {}})
            
            # 8. 원문 자막 (토글)
            original_chunks = [original_content[i:i+2000] for i in range(0, len(original_content), 2000)]
            original_blocks = []
            
            for chunk in original_chunks:
                original_blocks.append({
                    "object": "block",
                    "type": "paragraph",
                    "paragraph": {
                        "rich_text": [{"type": "text", "text": {"content": chunk}}]
                    }
                })
            
            blocks.append({
                "object": "block",
                "type": "toggle",
                "toggle": {
                    "rich_text": [{"type": "text", "text": {"content": "📄 원문 자막 보기"}}],
                    "children": original_blocks[:50]
                }
            })
            
            # 블록 추가
            blocks_response = requests.patch(
                f"https://api.notion.com/v1/blocks/{page_id}/children",
                headers=headers,
                data=json.dumps({"children": blocks})
            )
            
            if blocks_response.status_code != 200:
                print(f"⚠️ 콘텐츠 추가 부분 실패: {blocks_response.text}")
            else:
                print("✅ Notion 페이지 콘텐츠 추가 완료!")
            
            return {
                'success': True,
                'page_url': page_url,
                'page_id': page_id,
                'error': None
            }
            
        except Exception as e:
            print(f"❌ Notion 페이지 생성 오류: {str(e)}")
            return {
                'success': False,
                'error': f'Notion 페이지 생성 중 오류: {str(e)}'
            }
    
    async def process_single_video(self, url: str, custom_prompt: str = '', 
                                 custom_title: str = '') -> Dict[str, any]:
        """
        단일 비디오 처리 (모드 1)
        """
        print(f"📺 모드 1: 직접 URL 처리 - {url}")
        
        result = {
            'mode': 'single_video',
            'url': url,
            'transcript_result': {},
            'summary_result': {},
            'notion_result': {},
            'overall_success': False,
            'errors': []
        }
        
        try:
            # 1. 자막 추출
            print("\n📺 1단계: YouTube 자막 추출...")
            transcript_result = self.extract_youtube_transcript(url)
            result['transcript_result'] = transcript_result
            
            if not transcript_result['success']:
                result['errors'].append(transcript_result['error'])
                return result
            
            # 2. Gemini 요약
            print("\n🧠 2단계: Gemini AI 요약 생성...")
            summary_result = await self.summarize_with_gemini(
                transcript_result['content'], 
                custom_prompt
            )
            result['summary_result'] = summary_result
            
            if not summary_result['success']:
                result['errors'].append(summary_result['error'])
                return result
            
            # 3. Notion 저장
            print("\n📝 3단계: Notion 페이지 생성...")
            title = custom_title if custom_title else f"YouTube 요약: {transcript_result.get('video_id', 'Unknown')}"
            
            metadata = {
                'language': transcript_result.get('language', 'unknown'),
                'length': transcript_result.get('length', 0),
                'video_id': transcript_result.get('video_id', ''),
                'tokens_used': summary_result.get('tokens_used', 0)
            }
            
            notion_result = self.create_notion_page_with_summary(
                title=title,
                summary=summary_result['summary'],
                original_content=transcript_result['content'],
                url=url,
                metadata=metadata
            )
            result['notion_result'] = notion_result
            
            if not notion_result['success']:
                result['errors'].append(notion_result['error'])
                return result
            
            result['overall_success'] = True
            print(f"\n🎉 단일 비디오 처리 완료! 페이지: {notion_result['page_url']}")
            
        except Exception as e:
            error_msg = f"단일 비디오 처리 중 오류: {str(e)}"
            result['errors'].append(error_msg)
            print(f"❌ {error_msg}")
        
        return result
    
    async def process_search_results(self, keywords: List[str], videos_per_keyword: int = 3, 
                                   custom_prompt: str = '') -> Dict[str, any]:
        """
        검색 결과 일괄 처리 (모드 2)
        
        Args:
            keywords: 검색어 리스트 (예: ["파이썬", "C", "자바"])
            videos_per_keyword: 키워드당 처리할 동영상 수
            custom_prompt: 사용자 정의 요약 프롬프트
            
        Returns:
            Dict: 전체 처리 결과
        """
        if not self.searcher or not self.searcher.youtube:
            return {
                'mode': 'search_batch',
                'overall_success': False,
                'errors': ['YouTube API 키가 설정되지 않아 검색 기능을 사용할 수 없습니다.']
            }
        
        print(f"🎯 모드 2: 검색어 일괄 처리 - {keywords}")
        
        result = {
            'mode': 'search_batch',
            'keywords': keywords,
            'videos_per_keyword': videos_per_keyword,
            'search_results': {},
            'processed_videos': {},
            'notion_pages': [],
            'overall_success': False,
            'errors': [],
            'summary_stats': {
                'total_searched': 0,
                'total_processed': 0,
                'successful_summaries': 0,
                'failed_summaries': 0
            }
        }
        
        try:
            # 1. 검색 단계
            print("\n🔍 1단계: YouTube 검색...")
            search_results = self.searcher.search_multiple_keywords(keywords, videos_per_keyword)
            result['search_results'] = search_results
            
            # 검색 결과 통계
            total_found = sum(len(videos) for videos in search_results.values())
            result['summary_stats']['total_searched'] = total_found
            
            if total_found == 0:
                result['errors'].append('검색 결과가 없습니다.')
                return result
            
            print(f"📊 총 {total_found}개의 동영상을 찾았습니다.")
            
            # 2. 각 비디오 처리
            print("\n🎬 2단계: 동영상별 자막 추출 및 요약...")
            
            for keyword, videos in search_results.items():
                print(f"\n🎯 키워드 '{keyword}' 처리 중...")
                result['processed_videos'][keyword] = []
                
                for i, video_info in enumerate(videos, 1):
                    print(f"\n📺 {keyword} - {i}/{len(videos)}: {video_info['title']}")
                    
                    video_result = {
                        'video_info': video_info,
                        'transcript_result': {},
                        'summary_result': {},
                        'notion_result': {},
                        'success': False
                    }
                    
                    try:
                        # 자막 추출
                        transcript_result = self.extract_youtube_transcript(video_info['video_id'])
                        video_result['transcript_result'] = transcript_result
                        
                        if not transcript_result['success']:
                            print(f"❌ 자막 추출 실패: {transcript_result['error']}")
                            result['summary_stats']['failed_summaries'] += 1
                            result['processed_videos'][keyword].append(video_result)
                            continue
                        
                        # Gemini 요약
                        summary_result = await self.summarize_with_gemini(
                            transcript_result['content'], 
                            custom_prompt,
                            video_info
                        )
                        video_result['summary_result'] = summary_result
                        
                        if not summary_result['success']:
                            print(f"❌ 요약 생성 실패: {summary_result['error']}")
                            result['summary_stats']['failed_summaries'] += 1
                            result['processed_videos'][keyword].append(video_result)
                            continue
                        
                        # Notion 저장
                        title = f"[{keyword}] {video_info['title']}"
                        metadata = {
                            'language': transcript_result.get('language', 'unknown'),
                            'length': transcript_result.get('length', 0),
                            'video_id': transcript_result.get('video_id', ''),
                            'tokens_used': summary_result.get('tokens_used', 0),
                            'keyword': keyword
                        }
                        
                        notion_result = self.create_notion_page_with_summary(
                            title=title,
                            summary=summary_result['summary'],
                            original_content=transcript_result['content'],
                            url=video_info['url'],
                            metadata=metadata,
                            video_info=video_info
                        )
                        video_result['notion_result'] = notion_result
                        
                        if notion_result['success']:
                            video_result['success'] = True
                            result['notion_pages'].append({
                                'keyword': keyword,
                                'title': video_info['title'],
                                'url': notion_result['page_url']
                            })
                            result['summary_stats']['successful_summaries'] += 1
                            print(f"✅ 처리 완료: {notion_result['page_url']}")
                        else:
                            print(f"❌ Notion 저장 실패: {notion_result['error']}")
                            result['summary_stats']['failed_summaries'] += 1
                        
                        result['summary_stats']['total_processed'] += 1
                        
                    except Exception as e:
                        print(f"❌ 비디오 처리 중 오류: {str(e)}")
                        result['summary_stats']['failed_summaries'] += 1
                    
                    result['processed_videos'][keyword].append(video_result)
                    
                    # API 제한을 위한 잠시 대기
                    await asyncio.sleep(2)
            
            # 3. 결과 정리
            if result['summary_stats']['successful_summaries'] > 0:
                result['overall_success'] = True
                print(f"\n🎉 일괄 처리 완료!")
                print(f"✅ 성공: {result['summary_stats']['successful_summaries']}개")
                print(f"❌ 실패: {result['summary_stats']['failed_summaries']}개")
            else:
                result['errors'].append('모든 비디오 처리에 실패했습니다.')
                
        except Exception as e:
            error_msg = f"일괄 처리 중 오류: {str(e)}"
            result['errors'].append(error_msg)
            print(f"❌ {error_msg}")
        
        return result

print("🤖 EnhancedYouTubeGeminiNotionAgent 클래스 정의 완료!")

🤖 EnhancedYouTubeGeminiNotionAgent 클래스 정의 완료!


## 🚀 Enhanced Agent 초기화

In [8]:
# Enhanced Agent 초기화
if all([NOTION_TOKEN, PARENT_PAGE_ID, GEMINI_API_KEY]):
    agent = EnhancedYouTubeGeminiNotionAgent(
        notion_token=NOTION_TOKEN,
        parent_page_id=PARENT_PAGE_ID,
        google_ai_api_key=GEMINI_API_KEY,
        youtube_api_key=YOUTUBE_API_KEY,  # 검색 기능용
        database_id=NOTION_DATABASE_ID
    )
    print("✅ Enhanced Agent 초기화 성공!")
else:
    print("❌ 필요한 API 키가 설정되지 않았습니다!")
    print("💡 위의 환경 설정 셀을 먼저 확인해주세요.")

2025-11-14 12:50:34,602 - googleapiclient.discovery_cache - INFO - file_cache is only supported with oauth2client<4.0.0


✅ YouTube Data API v3 초기화 성공!
🚀 Enhanced YouTube-Gemini-Notion Agent 초기화 완료!
✅ Enhanced Agent 초기화 성공!


## 🎯 모드 선택 및 실행

In [ ]:
# 모드 선택
print("🎯 처리 모드를 선택하세요:")
print("  1️⃣ 직접 URL 입력 (기존 방식)")
print("  2️⃣ 검색어로 인기 동영상 자동 선별 + 일괄 요약")

mode = input("\n모드 선택 (1 또는 2): ").strip()

if mode == "1":
    print("\n📺 모드 1: 직접 URL 입력 모드 선택됨")
    processing_mode = "single"
elif mode == "2":
    print("\n🎯 모드 2: 검색어 일괄 처리 모드 선택됨")
    processing_mode = "search"
else:
    print("\n📺 기본값으로 모드 1 선택됨")
    processing_mode = "single"

🎯 처리 모드를 선택하세요:
  1️⃣ 직접 URL 입력 (기존 방식)
  2️⃣ 검색어로 인기 동영상 자동 선별 + 일괄 요약

🎯 모드 2: 검색어 일괄 처리 모드 선택됨


In [ ]:
# 모드별 입력 처리
if processing_mode == "single":
    # 📺 모드 1: 직접 URL 입력
    print("\n🔗 요약할 YouTube URL을 입력하세요:")
    youtube_url = input("URL: ").strip()
    
    if not youtube_url:
        youtube_url = "https://youtu.be/6hSjOCUmjGQ?si=ES5FOYHaFjWtguUl"
        print(f"📺 기본 URL 사용: {youtube_url}")
    
    print("\n💬 사용자 정의 요약 프롬프트 (엔터 시 기본값):")
    custom_prompt = input().strip()
    
    print("\n📝 사용자 정의 페이지 제목 (엔터 시 자동 생성):")
    custom_title = input().strip()

elif processing_mode == "search":
    # 🎯 모드 2: 검색어 일괄 처리
    print("\n🔍 검색어를 입력하세요 (쉼표로 구분):")
    print("   예시: 파이썬, C, 자바")
    keywords_input = input("검색어: ").strip()
    
    if not keywords_input:
        keywords = ["파이썬", "C", "자바"]
        print(f"🎯 기본 검색어 사용: {keywords}")
    else:
        keywords = [k.strip() for k in keywords_input.split(',') if k.strip()]
        print(f"🎯 검색어: {keywords}")
    
    print("\n📊 각 검색어당 몇 개의 동영상을 처리할까요? (기본값: 3)")
    videos_input = input("동영상 수: ").strip()
    
    try:
        videos_per_keyword = int(videos_input) if videos_input else 3
    except ValueError:
        videos_per_keyword = 3
    
    print(f"📊 각 검색어당 {videos_per_keyword}개씩 처리 예정")
    
    print("\n💬 사용자 정의 요약 프롬프트 (엔터 시 기본값):")
    custom_prompt = input().strip()

print("\n📋 설정 완료!")


🔍 검색어를 입력하세요 (쉼표로 구분):
   예시: 파이썬, C, 자바
🎯 검색어: ['파이썬']

📊 각 검색어당 몇 개의 동영상을 처리할까요? (기본값: 3)
📊 각 검색어당 3개씩 처리 예정

💬 사용자 정의 요약 프롬프트 (엔터 시 기본값):

📋 설정 완료!


In [12]:
# 🚀 실행!
if all([NOTION_TOKEN, PARENT_PAGE_ID, GEMINI_API_KEY]):
    
    if processing_mode == "single":
        # 📺 모드 1: 단일 비디오 처리
        print("\n" + "🚀 모드 1: 단일 YouTube 비디오 처리 시작!".center(80, "="))
        
        result = await agent.process_single_video(
            url=youtube_url,
            custom_prompt=custom_prompt,
            custom_title=custom_title
        )
        
        # 결과 출력
        print("\n" + "📊 처리 결과".center(80, "="))
        
        if result['overall_success']:
            print("✅ 처리 성공!")
            print(f"📄 생성된 Notion 페이지: {result['notion_result']['page_url']}")
            
            # 요약 미리보기
            summary = result['summary_result'].get('summary', '')
            preview = summary[:200] + "..." if len(summary) > 200 else summary
            print(f"\n📋 요약 미리보기:\n{preview}")
        else:
            print("❌ 처리 실패:")
            for error in result['errors']:
                print(f"  • {error}")
    
    elif processing_mode == "search":
        # 🎯 모드 2: 검색 일괄 처리
        print("\n" + "🚀 모드 2: 검색어 일괄 처리 시작!".center(80, "="))
        print(f"🎯 검색어: {keywords}")
        print(f"📊 각 키워드당 {videos_per_keyword}개씩 처리")
        
        result = await agent.process_search_results(
            keywords=keywords,
            videos_per_keyword=videos_per_keyword,
            custom_prompt=custom_prompt
        )
        
        # 결과 출력
        print("\n" + "📊 일괄 처리 결과".center(80, "="))
        
        if result['overall_success']:
            stats = result['summary_stats']
            print(f"✅ 일괄 처리 완료!")
            print(f"📊 총 검색: {stats['total_searched']}개")
            print(f"✅ 성공: {stats['successful_summaries']}개")
            print(f"❌ 실패: {stats['failed_summaries']}개")
            
            # 생성된 Notion 페이지들
            if result['notion_pages']:
                print("\n📄 생성된 Notion 페이지들:")
                for page in result['notion_pages']:
                    print(f"  🎯 [{page['keyword']}] {page['title']}")
                    print(f"     {page['url']}")
        else:
            print("❌ 일괄 처리 실패:")
            for error in result['errors']:
                print(f"  • {error}")
            
            if result.get('summary_stats', {}).get('failed_summaries', 0) > 0:
                print("\n💡 일부 동영상에서 자막을 찾을 수 없거나 처리에 실패했습니다.")
                print("   자막이 있는 다른 키워드로 시도해보세요.")
    
    print("\n" + "="*80)
else:
    print("❌ 필요한 API 키가 설정되지 않았습니다!")


=============================🚀 모드 2: 검색어 일괄 처리 시작!==============================
🎯 검색어: ['파이썬']
📊 각 키워드당 3개씩 처리
🎯 모드 2: 검색어 일괄 처리 - ['파이썬']

🔍 1단계: YouTube 검색...

🎯 키워드 '파이썬' 처리 중...
🔍 '파이썬' 검색 중... (최대 3개)
✅ 3개의 인기 동영상을 찾았습니다!
  1. Introduction to Programming & Python | Python Tutorial - Day #1 (조회수: 10,605,927)
  2. Python OOP Tutorial 1: Classes and Instances (조회수: 4,826,801)
  3. Some Amazing Python Programs - The Power of Python | Python Tutorial - Day #2 (조회수: 3,404,976)
📊 총 3개의 동영상을 찾았습니다.

🎬 2단계: 동영상별 자막 추출 및 요약...

🎯 키워드 '파이썬' 처리 중...

📺 파이썬 - 1/3: Introduction to Programming & Python | Python Tutorial - Day #1
🔍 비디오 ID: 7wnove7K-ZQ
🔄 ko 언어 자막 시도 중...
❌ ko 실패: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=7wnove7K-ZQ! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['ko']

For this video (7wnove7K-ZQ) transcripts are available in the following languages:

(MANUALLY CREATED)
 - en-IN ("English 

## 📋 완료!

### 🎉 YouTube 검색 + AI 요약 + Notion 연동 완전판!

**🔥 두 가지 모드 지원:**

#### 📺 모드 1: 직접 URL 입력
- ✅ 기존 방식 그대로
- ✅ URL 하나씩 정확하게 처리
- ✅ 사용자 정의 프롬프트 및 제목 지원

#### 🎯 모드 2: 검색어 일괄 처리 ⭐
- ✅ **YouTube Data API v3** 활용
- ✅ 검색어 입력 → 인기 동영상 자동 선별
- ✅ 조회수 기준 정렬로 품질 높은 콘텐츠 선택
- ✅ 자막 있는 영상만 필터링
- ✅ 여러 키워드 동시 처리
- ✅ 일괄 요약 및 Notion 자동 저장

### 🔑 필요한 API 키:
```env
# 필수
NOTION_TOKEN=your_notion_token
notion_parent_page=your_parent_page_id
GOOGLE_AI_API_KEY=your_google_ai_key

# 검색 기능용 (모드 2 사용 시)
YOUTUBE_API_KEY=your_youtube_data_api_v3_key
```

### 🎯 사용 예시:
**모드 2 검색어 입력:**
- `파이썬, C, 자바` → 각 언어별 인기 프로그래밍 강의 3개씩
- `머신러닝, 딥러닝, AI` → AI 관련 인기 영상들
- `React, Vue, Angular` → 프론트엔드 프레임워크 비교

**결과:**
- 자동으로 인기 동영상 검색 및 선별
- 자막 추출 + Gemini 요약
- 키워드별로 구조화된 Notion 페이지 생성

### 📊 특징:
- 🎯 **스마트 검색**: 조회수, 자막 유무, 언어 등 다양한 필터
- 🤖 **AI 요약**: 비디오 메타데이터와 함께 구조화된 요약
- 📝 **자동 정리**: 키워드별로 깔끔하게 분류된 Notion 페이지
- ⚡ **효율성**: 한 번에 여러 키워드의 인기 콘텐츠를 모두 정리

**이제 "파이썬, C, 자바"만 입력하면 각 언어의 인기 강의들이 자동으로 요약되어 Notion에 정리됩니다!** 🚀✨